In [14]:
import os
import gzip
from struct import Struct
import pandas as pd
import time


matrix_path = './alevin_matrices/'
output_path = './alevin_output/'



In [15]:
def get_cb_count():
    cb_count = dict()
    for dir_name in os.listdir(matrix_path):
        if dir_name[0] != '.':
            f = open(matrix_path + dir_name + "/"+dir_name+'_rows.txt', 'r')
            for line in f:
                line = line.strip()
                cb_count[line] = cb_count.get(line, 0)+1
                #if line in cb_count:
                #    cb_count[line] = cb_count[line] + 1
                #else:
                #    cb_count[line] = 1
            f.close()
    return cb_count

cb_count = get_cb_count()


In [16]:
from shutil import copyfileobj
def merge():
    start = time.time()
    with open(output_path + 'concatenated.bin.gz', 'wb') as F1:
        rows_file = open(output_path + 'concatenated_rows.txt', 'w')


        for dir_name in os.listdir(matrix_path):
            if dir_name[0] != '.':
                
                """
                gene_file = matrix_path + dir_name + "/"+dir_name+'_cols.txt'
                print('gene file', gene_file)
                gene_names = pd.read_table(gene_file, header=None)[0].values

                num_genes = len(gene_names)
                header_struct = Struct( "d" * num_genes)
                """
                rows = open(matrix_path + dir_name + '/'+dir_name+'_rows.txt', 'r')
                #matrix = gzip.open(matrix_path + dir_name + '/'+dir_name+'.gz', 'r')
                with open(matrix_path + dir_name + '/'+dir_name+'.gz', 'rb') as F2:
                      copyfileobj(F2, F1)

                for cb in rows:
                    ## Current Version : no decompression just concatenate all matrices 
                    """
                    count += 1
                    try:
                        cell_values = header_struct.unpack_from(matrix.read(header_struct.size))
                    except:
                        print('counted: ', count)
                        print('Error in unpack')
                        break
                    #print(cell_values)
                    """
                    cb = cb.strip()
                    if cb_count[cb] >0 and cb_count[cb] == 1:
                        #values_file.write(header_struct.pack(*cell_values))
                        rows_file.write(cb + '\n')
                    else:
                        #common_cb[cb].append(cell_values) ## first version (182 secs)
                        #values_file.write(str(cell_values) + '\n')  ## second version (100 sec)
                        #values_file.write(header_struct.pack(*cell_values)) ## third version (84. secs)
                        rows_file.write(cb+"_"+str(cb_count[cb])+ '\n')
                        cb_count[cb] -= 1

                rows.close()
                

                
        """ ### ideally was done to keep same rows together for easier compression. 
        for key, val in common_cb.items():
                index = 0
                for cell_values in val:
                    rows_file.write(key + '_' + str(index) + '\n')
                    values_file.write(str(cell_values) + '\n')
                    index += 1
        """
        
        rows_file.close()
    end = time.time()
    print('Finished. Time: ', end - start)

merge()

Finished. Time:  7.781069040298462
